<a href="https://colab.research.google.com/github/tony1swedberg/EMNIST_CNN/blob/main/EMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trains a convolutional neural network on the EMNIST ByClass dataset. The EMNIST ByClass dataset is a set of 814,255 handwritten characters and digits split into 62 classes.

In [ ]:
import math
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets         # For EMNIST dataset
import torchvision.transforms as transforms
import pandas as pd

In [ ]:
epoch_num = 5             # Lower for training purposes
batch_size = 100          #
learning_rate = .01       #

In [ ]:
if torch.cuda.is_available():
    print('We are utilizing the GPU')
else:
    print('We are using the CPU')

We are utilizing the GPU


In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

In [ ]:
#split can be byclass, bymerge, balanced, letters, digits, and mnist
trainset = datasets.EMNIST(root='./data', train=True, download=True, split='byclass', transform=transform)
testset = datasets.EMNIST(root='./data', train=False, download=True, split='byclass', transform=transform)

In [ ]:
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
#This cell is defining the structure of each CNN, NetOne is larger and takes much longer
#to run than NetTwo. We are using NetTwo currently

class NetOne(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 50, 5)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(50, 500, 3)
      self.conv3 = nn.Conv2d(500, 10000, 2)
      self.fc1 = nn.Linear(10000*2*2, 1000)
      self.fc2 = nn.Linear(1000, 500)
      self.fc3 = nn.Linear(500, 5000)
      self.fc4 = nn.Linear(5000, 62)

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = self.pool(x)
      x = F.relu(self.conv2(x))
      x = self.pool(x)
      x = F.relu(self.conv3(x))
      x = self.pool(x)
      x = torch.flatten(x, 1)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      x = self.fc4(x)
      return x

class NetTwo(nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = nn.Conv2d(1, 10, 5)
      self.pool = nn.MaxPool2d(2, 2)
      self.conv2 = nn.Conv2d(10, 25, 3)
      self.conv3 = nn.Conv2d(25, 50, 2)
      self.fc1 = nn.Linear(50*2*2, 100)
      self.fc2 = nn.Linear(100, 75)
      self.fc3 = nn.Linear(75, 150)
      self.fc4 = nn.Linear(150, 62)

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = self.pool(x)
      x = F.relu(self.conv2(x))
      x = self.pool(x)
      x = F.relu(self.conv3(x))
      x = self.pool(x)
      x = torch.flatten(x, 1)
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = F.relu(self.fc3(x))
      x = self.fc4(x)
      return x

net = NetTwo()

In [ ]:
#This cell defines an accuracy function for our CNN

def accuracy(net, loader):
    correct = 0
    total = 0
    for images, labels in loader:
        if torch.cuda.is_available():
            net = net.cuda()
            images = images.cuda()
            labels = labels.cuda()
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    return correct / total

In [ ]:
#This cell is where we do the training of the CNN

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=.9)
for epoch in range(1, epoch_num + 1):
    batch_num = 1
    for inputs, labels in train_loader:
        if torch.cuda.is_available():          #If the GPU is available we use it to speed up training
            net = net.cuda()
            inputs = inputs.cuda()
            labels = labels.cuda()

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if batch_num % 500 == 0:               #To see how quickly the model is training we print every 500 batches 
            print(f'We are on epoch {epoch} of {epoch_num}, batch {batch_num} of {len(train_loader)}')
        batch_num += 1
    train_accuracy = 100 * accuracy(net, train_loader)   #Calculate the train accuracy after each epoch
    test_accuracy = 100 * accuracy(net, test_loader)     #Calculate the test accuracy after each epoch
    print(f'For epoch {epoch} train accuracy is {train_accuracy}%, and test accuracy is {test_accuracy}%')
print(f'The final test accuracy of this convolutional neural network on EMNIST is {test_accuracy}%')

We are on epoch 1 of 5, batch 500 of 6980
We are on epoch 1 of 5, batch 1000 of 6980
We are on epoch 1 of 5, batch 1500 of 6980
We are on epoch 1 of 5, batch 2000 of 6980
We are on epoch 1 of 5, batch 2500 of 6980
We are on epoch 1 of 5, batch 3000 of 6980
We are on epoch 1 of 5, batch 3500 of 6980
We are on epoch 1 of 5, batch 4000 of 6980
We are on epoch 1 of 5, batch 4500 of 6980
We are on epoch 1 of 5, batch 5000 of 6980
We are on epoch 1 of 5, batch 5500 of 6980
We are on epoch 1 of 5, batch 6000 of 6980
We are on epoch 1 of 5, batch 6500 of 6980
For epoch 1 train accuracy is 84.4196855854152%, and test accuracy is 84.31350635729822%
We are on epoch 2 of 5, batch 500 of 6980
We are on epoch 2 of 5, batch 1000 of 6980
We are on epoch 2 of 5, batch 1500 of 6980
We are on epoch 2 of 5, batch 2000 of 6980
We are on epoch 2 of 5, batch 2500 of 6980
We are on epoch 2 of 5, batch 3000 of 6980
We are on epoch 2 of 5, batch 3500 of 6980
We are on epoch 2 of 5, batch 4000 of 6980
We are on 